In [ ]:
#steps

# Import large Mt5 model
# bring in dataset
# Fine tune
# Evaluate

In [53]:
##Bring in dataset

from datasets import load_dataset

dataset = load_dataset("squad_es",'v1.1.0')


Reusing dataset squad_es (/root/.cache/huggingface/datasets/squad_es/v1.1.0/1.1.0/bcada4f600192451443b95e24f609325705c5185b8aad97bffa8bc3784a867ad)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87595
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

## Data Conversion

In [63]:
# Step 1 and 2: Combine context and question, format the answer

# Start with a subset of your data for testing
train_subset = dataset['train'][:10]  # Adjust this as needed

# Create new lists for our inputs and answers
inputs = []
answers = []

# Iterate over each item in the train_subset data
for id_value, title_value, context_value in zip(train_subset['id'], train_subset['title'], train_subset['context']):
    # Combine the context and question into a single string
    input_str = "context: " + context_value + " question: " + title_value
    inputs.append(input_str)
    
    print("id value",id_value)

id value 5733be284776f41900661182
id value 5733be284776f4190066117f
id value 5733be284776f41900661180
id value 5733be284776f41900661181
id value 5733be284776f4190066117e
id value 5733bf84d058e614000b61be
id value 5733bf84d058e614000b61bf
id value 5733bf84d058e614000b61c0
id value 5733bf84d058e614000b61bd
id value 5733bf84d058e614000b61c1


In [64]:
train_subset

{'id': ['5733be284776f41900661182',
  '5733be284776f4190066117f',
  '5733be284776f41900661180',
  '5733be284776f41900661181',
  '5733be284776f4190066117e',
  '5733bf84d058e614000b61be',
  '5733bf84d058e614000b61bf',
  '5733bf84d058e614000b61c0',
  '5733bf84d058e614000b61bd',
  '5733bf84d058e614000b61c1'],
 'title': ['Universidad _ de _ Notre _ Dame',
  'Universidad _ de _ Notre _ Dame',
  'Universidad _ de _ Notre _ Dame',
  'Universidad _ de _ Notre _ Dame',
  'Universidad _ de _ Notre _ Dame',
  'Universidad _ de _ Notre _ Dame',
  'Universidad _ de _ Notre _ Dame',
  'Universidad _ de _ Notre _ Dame',
  'Universidad _ de _ Notre _ Dame',
  'Universidad _ de _ Notre _ Dame'],
 'context': ['Arquitectónicamente, la escuela tiene un carácter católico. Encima de la cúpula de oro del edificio principal hay una estatua dorada de la Virgen María. Inmediatamente delante del edificio principal y frente a él, hay una estatua de cobre de Cristo con los brazos levantados con la leyenda "Venite A

In [72]:

# Step 1 and 2: Combine context and question, format the answer
from transformers import T5TokenizerFast

# Load the tokenizer
tokenizer = T5TokenizerFast.from_pretrained('google/mt5-base')

# Start with a subset of your data for testing
train_subset = dataset['train'][:10]  # Adjust this as needed

# Create new lists for our inputs and answers
input_ids = []
answers = []

for id_value, title_value, context_value, q_value, a_value in zip(train_subset['id'], train_subset['title'], train_subset['context'],train_subset['question'],train_subset['answers']):
    
    
    # Combine the context and question into a single string
    input_str = context_value + " " + q_value
    #print(input_str)
    
    #tokenize the inputs
    inputs = tokenizer(input_str, truncation=True, padding='max_length', max_length=512)
    #print(inputs.input_ids)
    input_ids.append(inputs.input_ids)
    
    
    

    # Format the answer (assuming there's always exactly one answer)
    #answer_str = example['answers']['text'][0]
    #answers.append(answer_str)
    

# Check the first few examples to make sure everything looks good
for i in range(5):
    print(f"Input: {input_ids[i]}")
    #print(f"Answer: {answers[i]}\n")


loading file https://huggingface.co/google/mt5-base/resolve/main/spiece.model from cache at /root/.cache/huggingface/transformers/4764ec347af4d2d6286acbe1d9d630ac0afd8554a4c4a64170e0b663fd2e2412.84ea7af2df68dc8db434d3160aab65cce8ac63ce5b6f7743f8c9a4a14b4f77e2
loading file https://huggingface.co/google/mt5-base/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/google/mt5-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/google/mt5-base/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/0d7d5b3fc19bf58d4b274990c8bcf5e307726bc18d95f40a1436dfb6a0892f85.294ebaa4cd17bb284635004c92d2c4d522ec488c828dcce0c2471b6f28e3fe82
loading file https://huggingface.co/google/mt5-base/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/afba33be693521ccefbde6d03b93b5c517d7108ba31f6c08000ed52c2cea45c9.28bbf90ae7962b1b7211c0ce8b2006f968c82439ec9c47e0847ba63642f9435a

Input: [259, 220933, 1790, 32294, 261, 283, 259, 60876, 2812, 335, 259, 58878, 41742, 10292, 108405, 260, 642, 22472, 269, 283, 37795, 54126, 269, 16347, 426, 259, 39909, 268, 6017, 2318, 573, 99834, 262, 123281, 262, 269, 283, 89306, 278, 259, 24952, 260, 563, 87060, 259, 63047, 426, 259, 39909, 268, 6017, 259, 276, 259, 20189, 259, 262, 7938, 261, 2318, 573, 99834, 262, 269, 56333, 265, 269, 39333, 450, 595, 259, 98838, 263, 57692, 2131, 450, 283, 340, 116073, 313, 30702, 15782, 4515, 1517, 3548, 1865, 1191, 4143, 476, 440, 259, 39909, 268, 6017, 1957, 283, 364, 7590, 33858, 426, 1560, 15905, 7090, 56019, 260, 563, 87060, 269, 70774, 269, 283, 330, 7590, 33858, 1957, 283, 27166, 422, 261, 335, 5967, 207462, 268, 269, 259, 211457, 259, 276, 130784, 1790, 260, 1659, 573, 1888, 94336, 269, 283, 10937, 422, 269, 458, 133796, 261, 39233, 261, 259, 7497, 283, 89306, 278, 259, 24952, 259, 216207, 1664, 303, 340, 44406, 2711, 259, 262, 5909, 60442, 4549, 16856, 7377, 6836, 289, 259, 116096, 

In [24]:
# Extract one example from each tensor
sample = {
    'input_ids': train_data['input_ids'][0],
    'attention_mask': train_data['attention_mask'][0],
    'labels': train_data['labels'][0]
}

assert isinstance(sample, dict), "Each item in dataset must be a dictionary"
assert 'input_ids' in sample, "input_ids must be in the dataset"
assert 'attention_mask' in sample, "attention_mask must be in the dataset"
assert 'labels' in sample, "labels must be in the dataset"

print("An example in your dataset appears to be correctly structured.")


An example in your dataset appears to be correctly structured.


In [51]:
# Convert train_data from dict of lists to list of dicts
#validation_data = [dict(zip(validation_data, t)) for t in zip(*validation_data.values())]

# Now train_data[0] should work, let's print it
#print(validation_data[0])

In [52]:
# Convert train_data from dict of lists to list of dicts
#train_data = [dict(zip(train_data, t)) for t in zip(*train_data.values())]

# Now train_data[0] should work, let's print it
#print(train_data[0])


## Data ready test

In [28]:
# replace 'train_dataset' with your dataset variable
sample = validation_data[0]

assert isinstance(sample, dict), "Each item in dataset must be a dictionary"
assert 'input_ids' in sample, "input_ids must be in the dataset"
assert 'attention_mask' in sample, "attention_mask must be in the dataset"
assert 'labels' in sample, "labels must be in the dataset"

print("Your dataset appears to be correctly structured for Trainer.")


Your dataset appears to be correctly structured for Trainer.


## Fine Tuning

In [29]:
from transformers import MT5ForConditionalGeneration, Trainer, TrainingArguments

# Initialize the model
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-base")

# Initialize the training arguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Define the compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Initialize the Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_data,            # training dataset
    eval_dataset=validation_data,         # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

# Train the model
trainer.train()


loading configuration file https://huggingface.co/google/mt5-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/5ebfd830555547194403d6803baa127970de59b443c04b7a1a60b16a97ed3958.b589da7dac64196f9764abaf2c4c7e507cec8b14b96da3ef270d924f155062de
Model config MT5Config {
  "_name_or_path": "/home/patrick/hugging_face/t5/mt5-base",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Token

Step,Training Loss
10,51.877200




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=12, training_loss=51.85478909810384, metrics={'train_runtime': 11.0849, 'train_samples_per_second': 17.321, 'train_steps_per_second': 1.083, 'total_flos': 230216857288704.0, 'train_loss': 51.85478909810384, 'epoch': 3.0})